# 🎙️ ASR Project Demo (CTC-BiLSTM)

Демонстрация работы системы распознавания речи.


## 1. 🛠️ Установка и Скачивание кода

In [ ]:
import os
USER = 'Olga57'
REPO = 'speech-recognition-ctc'

if not os.path.exists(REPO):
    !git clone https://github.com/{USER}/{REPO}.git

%cd {REPO}
!pip install -q -r requirements.txt
!pip install -q gdown hydra-core omegaconf
!apt-get install -y libsndfile1 ffmpeg > /dev/null

## 2. ⚡ Быстрый тест (Librispeech)

In [ ]:
import os
import sys

# 1. Настраиваем пути (чтобы видеть src)
project_root = os.getcwd()
os.environ['PYTHONPATH'] = project_root

# 2. ЗАПУСК (С добавлением train_source=null)
# Мы используем +, чтобы добавить параметры, которых нет в конфиге
!python inference.py dataset=librispeech \
    +dataset.valid_split=validation.clean \
    +dataset.max_valid_items=5 \
    +dataset.train_source=null \
    +device='cuda' \
    +decode='beam' \
    +out_dir='predictions_libri'

!head -n 5 predictions_libri/*.txt

## 3. 📂 Тест на вашем датасете (Google Drive)

In [ ]:
import gdown
import os
os.environ['PYTHONPATH'] = os.getcwd()
url = input('Ссылка на Google Drive (ZIP): ')
if 'drive.google.com' in url:
    file_id = url.split('/d/')[-1].split('/')[0]
    dl_url = f'https://drive.google.com/uc?id={file_id}'
    gdown.download(dl_url, 'custom.zip', quiet=False)
    !unzip -q -o custom.zip -d custom_data
    ds_root = 'custom_data'
    for r, d, f in os.walk('custom_data'):
        if 'audio' in d: ds_root = r
    # Тут тоже добавляем train_source=null
    !python inference.py dataset=custom dataset.custom_root="{ds_root}" \
         +device='cuda' +decode='beam' +out_dir='preds_custom' +dataset.train_source=null
    !python calc_metrics.py --ref_dir "{ds_root}/transcriptions" --hyp_dir "preds_custom"
else:
    print('Это не ссылка на Google Drive')